# inelasty

We begin by importing some necessary packages

In [ ]:
from ase.io import read
import sys
import glob
import re

# This path should be replaced with the relevant path to your clone of the inelasty repo
sys.path.append("/home/b55k/harryrich11.b55k/INS")
from inelasty_data import inelasty

We then define three crucial variables for the HPC environment: `vasp_file_path`, the path to the VASP installation (used to set `VASP_PP_PATH` and locate `vasp_std`); `conda_command`, which activates the Conda environment containing ASE and vasp; and `spack_command`, which loads the required VASP runtime dependencies via Spack.


In [ ]:
vasp_file_path = "/home/b55k/harryrich11.b55k/vasp"
conda_command = "source ~/miniforge3/bin/activate vasp"
spack_command = """. ~/spack/share/spack/setup-env.sh 
spack env activate vasp_fin"""

We also import several dictionaries from `settings.py`. These dictionaries define groups of VASP input parameters (passed directly to the ASE `Vasp` calculator) tailored for different stages of the workflow, such as geometry relaxation, k-point convergence, ENCUT convergence, and phonon force calculations.


In [ ]:
from settings import (
    kpoint_kwargs,
    encut_kwargs,
    geom_relax_kwargs1,
    geom_relax_kwargs2,
    phonopy_kwargs,
)

For example, `kpoint_kwargs` contains a standard set of VASP parameters for k-point convergence studies:

In [ ]:
kpoint_kwargs

We can now instantiate the `inelasty` class for the crystal of interest. The structure is read using ASE and passed to `inelasty` along with the previously defined HPC environment configuration.


In [ ]:
benz_cryst = "/home/b55k/harryrich11.b55k/benz_cryst_vasp_prod/benzene_solid.cif"
atoms = read(benz_cryst)
benz = inelasty(
    struc=atoms,
    vasp_path=vasp_file_path,
    conda_command=conda_command,
    spack_command=spack_command,
)

We can now begin using the core functionality of `inelasty`. The workflow is driven by a small number of high-level methods corresponding to common VASP tasks: geometry relaxation, k-point convergence, ENCUT convergence, and phonon calculations.

The `relax` method performs a two-stage geometry optimisation using separate sets of VASP parameters. The walltime and parallelisation settings (`time`, `ntasks_per_node`) control the Slurm submission, while all electronic and ionic parameters are supplied via the VASP keyword dictionaries.

In all cases, calculation-specific behaviour is controlled through the VASP keyword dictionaries, while resource usage and directory structure are by default handled automatically by `inelasty`.

For more information see the function doc strings

In [ ]:
benz.relax(
    vasp_kwargs1=geom_relax_kwargs1, vasp_kwargs2=geom_relax_kwargs2, time="08:00:00"
)

Subsequent steps follow the same pattern:
- `run_k_points(...)` performs single-point calculations for a list of k-point meshes.
- `run_encut(...)` evaluates convergence with respect to the plane-wave cutoff.
- `generate_phonopy(...)` generates displaced supercells and optionally submits force calculations for phonon analysis.

We can now instantiate the class again based on the optimised structure produced by the relaxation and use it to test k-point and encut convergence:

In [ ]:
relaxed_struc = (
    "/home/b55k/harryrich11.b55k/benz_cryst_vasp_prod/geom_opt/stage_2/CONTCAR"
)
atoms = read(relaxed_struc)
benz = inelasty(
    struc=atoms,
    vasp_path=vasp_file_path,
    conda_command=conda_command,
    spack_command=spack_command,
)

In [ ]:
kpoints_list = [(1, 1, 1), (2, 2, 2), (3, 3, 3)]
benz.run_k_points(kpoint_kwargs=kpoint_kwargs, kpoints_list=kpoints_list)

In [ ]:
encut_list = [100, 200, 300, 400, 500, 600, 700, 800]
benz.run_encut(vasp_kwargs=encut_kwargs, encut_list=encut_list)

Finally, we use `generate_phonopy` to set up and run the phonon force calculations required by Phonopy.

This method generates a set of displaced supercells from the relaxed structure, writes the corresponding POSCAR files, and (when `run=True`) submits independent VASP force calculations for each displacement.

Key arguments control both the physics and the HPC resources:
- `atom_path` specifies the relaxed unit cell used to build the supercells.
- `vasp_kwargs` defines the VASP settings used to compute forces.
- `supercell_size` controls the size of the phonon supercell.
- `run` toggles submission of force calculations to HPC.
- `ntasks_per_node` and `time` define the Slurm resource allocation.

In [ ]:
benz.generate_phonopy(
    atom_path=relaxed_struc,
    vasp_kwargs=phonopy_kwargs,
    run=True,
    supercell_size=2,
    ntasks_per_node=24,
    time="24:00:00",
)

From the resulting calculations we can then use phonopy to aggregate and produce the force constants. The force constants file forms the input for abINS for the INS spectra production.

In [ ]:
import phonopy
from phonopy.interface.vasp import parse_set_of_forces

phonon = phonopy.load("phonopy/phonopy_disp.yaml")
outcar_files = sorted(glob.glob("./phonopy/phonopy-*/vasprun.xml"))
# shamelessly stolen from stack overflow sorting regex
outcar_files.sort(
    key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)]
)

sets_of_forces = parse_set_of_forces((12 * 4 * 2**3), outcar_files)
phonon.forces = sets_of_forces["forces"]
phonon.produce_force_constants()
phonon.save(settings={"force_constants": True})